In [1]:
import os
import numpy as np
import sklearn as sk

In [2]:
def get_count_trajectory(fname, cache_fname=None):
    if cache_fname and os.path.exists(cache_fname):
        return np.load(cache_fname)
    else:
        dtraj = []
        if os.path.exists(fname):
            with h5py.File(fname) as f:
                traj = f["readdy/trajectory"]
                traj_time = traj["time"]
                traj_time_records = traj["records"]
                for time, records in zip(traj_time, traj_time_records):
                    current_counts = [0]*3
                    for record in records:
                        type_id = record["typeId"]
                        current_counts[type_id] += 1
                    dtraj.append(current_counts)
        else:
            log.warn("file {} did not exist".format(fname))
        dtraj = np.array(dtraj)
        if cache_fname:
            np.save(cache_fname, dtraj)
        return dtraj

In [3]:
counts = get_count_trajectory("../generate/simple_trajectory.h5", cache_fname='counts_traj.npy')

In [4]:
dcounts_dt = np.gradient(counts, axis=0)

In [5]:
theta1 = lambda x: x[:,0]
theta2 = lambda x: x[:,1]
theta3 = lambda x: x[:,2]
theta4 = lambda x: x[:,0]*x[:,1]
theta5 = lambda x: x[:,0]*x[:,2]
theta6 = lambda x: x[:,1]*x[:,2]

In [6]:
theta = np.array([theta1(counts), theta2(counts), theta3(counts), theta4(counts), theta5(counts), theta6(counts)])

In [7]:
# theta in R^(species x ansatz functions * n_timesteps)
print(theta.shape)

(6, 50001)


In [8]:
xi = np.array([[.5, .5, .5, .5, .5, .5],[.5, .5, .5, .5, .5, .5],[.5, .5, .5, .5, .5, .5]])

In [13]:
xi.dot(theta).T.shape

(50001, 3)

In [51]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=15.0)

In [52]:
clf.fit(theta.T, dcounts_dt)

Lasso(alpha=15.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [53]:
print(dcounts_dt.shape)
print(theta.shape)

(50001, 3)
(6, 50001)


In [54]:
print(clf.coef_)

[[ -0.00000000e+00  -0.00000000e+00   0.00000000e+00  -1.76121056e-07
   -0.00000000e+00  -0.00000000e+00]
 [ -0.00000000e+00  -0.00000000e+00   0.00000000e+00  -1.76121056e-07
   -0.00000000e+00  -0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00  -0.00000000e+00   1.76121056e-07
    0.00000000e+00   0.00000000e+00]]
